# Advanced Jupyter Tips
https://towardsdatascience.com/learn-advanced-features-for-pythons-main-data-analysis-library-in-20-minutes-d0eedd90d086

In [2]:
# Make screen wider
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:90% !important;}</style>"))

In [3]:
import pandas as pd

In [23]:
invoices = pd.read_csv('https://raw.githubusercontent.com/FBosler/you-datascientist/master/invoices.csv')

In [20]:
invoices.info()
invoices.describe()
invoices.sample(5)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50017 entries, 0 to 50016
Data columns (total 9 columns):
Order Id             50017 non-null object
Date                 50017 non-null object
Meal Id              50017 non-null object
Company Id           50017 non-null object
Date of Meal         50017 non-null object
Participants         50017 non-null object
Meal Price           50017 non-null float64
Type of Meal         50017 non-null object
Heroes Adjustment    50017 non-null bool
dtypes: bool(1), float64(1), object(7)
memory usage: 3.1+ MB


,Order Id,Date,Meal Id,Company Id,Date of Meal,Participants,Meal Price,Type of Meal,Heroes Adjustment
48923,R0M0TC6KH91YLKDC,2018-09-30,NHXO3LCN02ILDTWA,1CGXEJLX3LBDR7YE,2018-09-29 07:00:00+02:00,['Gordon Shin'],737.0,Breakfast,False
37067,ELIL27C4ZBVF92RE,2014-12-17,T9JRJ6JRJUPN3ZMJ,0MXDIXQCPE6ZRV1F,2014-12-18 08:00:00+01:00,['Jose Clark' 'Debra Litke' 'Tony Ridge' 'Fran...,103.0,Breakfast,True
34924,7RKIU73SZHUDSISG,2018-09-01,WDFNZU28HQPC5AUK,H2DPBBCVWD7U967V,2018-08-28 14:00:00+02:00,['Patsy Pierce'],107.0,Lunch,False
38723,C5VNRZC1W3O1K2JW,2014-04-17,SW2OZD3GS43DOLIC,DWX6UI7GHZRJ82IC,2014-04-20 20:00:00+02:00,['James Mccoy'],47.0,Dinner,False
30992,X22FDPAR40B6HWBN,2016-10-24,X8Q1H2GSWGD3TVZL,3FWM3IGCFHUH4ANE,2016-10-24 22:00:00+02:00,['Madeline Love' 'Zenobia Collis'],468.0,Dinner,False


In [17]:
invoices.shape

(50017, 9)

In [18]:
invoices.dtypes

Order Id              object
Date                  object
Meal Id               object
Company Id            object
Date of Meal          object
Participants          object
Meal Price           float64
Type of Meal          object
Heroes Adjustment       bool
dtype: object

## 1. Converting Data Types
Let's make some the data types more explicit as Jupyter did its best but it could be better.
Two ways, astype() or pd.to_numeric/pd.to_datetime

In [24]:
invoices['Type of Meal'] = invoices['Type of Meal'].astype('category')
invoices['Date'] = invoices['Date'].astype('datetime64')
invoices['Meal Price'] = invoices['Meal Price'].astype('int64')

In [26]:
invoices.dtypes

Order Id                     object
Date                 datetime64[ns]
Meal Id                      object
Company Id                   object
Date of Meal                 object
Participants                 object
Meal Price                    int64
Type of Meal               category
Heroes Adjustment              bool
dtype: object

In [27]:
# For the sake of causing a bit of trouble in the data set. Changing these to strings will mean we cant convert to integers.
invoices.loc[45612,'Meal Price'] = 'I am causing trouble'
invoices.loc[35612,'Meal Price'] = 'Me too'

In [34]:
# "Whenever I encounter unexpected conversion errors, I typically check the values of the column explicitly to get a better understanding of the magnitude of the strange values."
invoices['Meal Price'].apply(lambda x: type(x)).value_counts()

<class 'int'>    50015
<class 'str'>        2
Name: Meal Price, dtype: int64

In [35]:
#You could then identify the offending rows by doing this:
invoices['Meal Price'][invoices['Meal Price'].apply(
  lambda x: isinstance(x,str)
)]

35612                  Me too
45612    I am causing trouble
Name: Meal Price, dtype: object

In [25]:
# Lets make those two troublesome strings Nan. errors='coerce' will convert them to Nan
# convert the offending values into np.nan
invoices['Meal Price'] = pd.to_numeric(invoices['Meal Price'],errors='coerce')
# fill np.nan with the median of the data
invoices['Meal Price'] = invoices['Meal Price'].fillna(invoices['Meal Price'].median())
#convert column to an integer
invoices['Meal Price'] = invoices['Meal Price'].astype('int64') #'int' will return int34 by default.

invoices['Meal Price'].apply(lambda x: type(x)).value_counts()

<class 'int'>    50017
Name: Meal Price, dtype: int64

In [26]:
# Lets convert the data of meal using pd.to_datetime
invoices['Date of Meal'] = pd.to_datetime(invoices['Date of Meal'], utc=True)
invoices.dtypes

Order Id                          object
Date                      datetime64[ns]
Meal Id                           object
Company Id                        object
Date of Meal         datetime64[ns, UTC]
Participants                      object
Meal Price                         int64
Type of Meal                    category
Heroes Adjustment                   bool
dtype: object

## 2. Useful accessor methods for certain data types
Three different accessors, dt, str and cat

In [45]:
#dt
invoices['Date of Meal'].dt.date
invoices['Date of Meal'].dt.weekday_name
invoices['Date of Meal'].dt.month_name
invoices['Date of Meal'].dt.day #etc etc
# dt.week, month, quarter, year, is_leap_year, to_py_datetime() to name some.
pass

test


In [47]:
#str
invoices['Type of Meal'].str.lower() #lower case
invoices['Type of Meal'].str.upper()
#str.ljust(width=X), rjust(width=X), center(width=X), zfill(width=X) where X is the desired value.
invoices['Type of Meal'].str.endswith('ast')
#str.startswith(), contains(), swapcase(), repeat(times)
invoices['Price'].str.replace('[\$\,\.]', '')
pass

test


In [51]:
#cat, categorical opertations
invoices['Type of Meal'].cat.ordered # returns false as column is not ordered
invoices['Type of Meal'].cat.categories # returns a list of categories.
invoices['Type of Meal'].cat.codes # returns numerical value of categories.
invoices['Type of Meal'].cat.reorder_categories(['Lunch', 'Breakfast', 'Dinner'])
pass

## Combining Dataframes
Concatenating vertically and horizontally, and merge()

In [59]:
invoices['Date of Meal'].dt.year.value_counts().sort_index()

2013       34
2014    10051
2015    10036
2016     9862
2017     9921
2018    10095
2019       18
Name: Date of Meal, dtype: int64

In [76]:
# Split the data . We use copy() to make sure that the dataframe is a copy and not just a reference to the dataframe.
y_2013 = invoices[invoices['Date of Meal'].dt.year == 2013].copy()
y_2014 = invoices[invoices['Date of Meal'].dt.year == 2014].copy()
y_2015 = invoices[invoices['Date of Meal'].dt.year == 2015].copy()
y_2016 = invoices[invoices['Date of Meal'].dt.year == 2016].copy()
y_2017 = invoices[invoices['Date of Meal'].dt.year == 2017].copy()
y_2018 = invoices[invoices['Date of Meal'].dt.year == 2018].copy()
y_2019 = invoices[invoices['Date of Meal'].dt.year == 2019].copy()


### Vertical concatenating / stacking

In [93]:
# keys adds an index for the individual dataframes that have been concatenated.
# names adds labels to the key index and the index from the individual concatenated dataframes.
invoices1 = pd.concat(
    [y_2013, y_2014, y_2015, y_2016, y_2017, y_2018, y_2019],
    keys=[13, 14, 15, 16, 17, 18, 19],
    names=['Year', 'Original_Index'])
pass

### Horizontal concatenating / stacking

In [103]:
import datetime
import numpy as np

range_a = pd.date_range(
    '20190102',
    '20190108'
)
df_a = pd.DataFrame(
    index=range_a, 
    data=np.random.randint(2,100,size=len(range_a)),
    columns=['observations_A']
)
range_b = pd.date_range(
    datetime.datetime(2019,1,5),
    datetime.datetime(2019,1,12) # I removed datetime.datetime above simply to achieve the same result another way.
)
df_b = pd.DataFrame(
    index=range_b, 
    data=np.random.randint(2,100,size=len(range_b)),
    columns=['observations_B']
)
pd.concat([df_a,df_b],axis=1)

,observations_A,observations_B
2019-01-02,56.0,NaN
2019-01-03,24.0,NaN
2019-01-04,11.0,NaN
2019-01-05,7.0,96.0
2019-01-06,78.0,36.0
2019-01-07,64.0,85.0
2019-01-08,69.0,23.0
2019-01-09,NaN,2.0
2019-01-10,NaN,82.0
2019-01-11,NaN,85.0


In [10]:
# Making my own DataFrame for giggles.
import numpy as np
import pandas as pd

range3 = range(1, 50)

df3 = pd.DataFrame(index=range3, 
                   data=np.random.randint(4000, 10000, 
                                          size=len(range3)),
                   columns=['observations'])
pass

### Merge()

In [18]:
# Load some additional Data
# Note the parse_dates? We need this to have consistent types with invoices, otherwise the subsequent merges would throw errors
order_data = pd.read_csv('https://raw.githubusercontent.com/FBosler/you-datascientist/master/order_leads.csv', parse_dates=[3])

In [12]:
sales_team = pd.read_csv('https://raw.githubusercontent.com/FBosler/you-datascientist/master/sales_team.csv')

In [35]:
order_data.head()
order_data.shape


(100000, 6)

In [21]:
sales_team.head()
#sales_team.dtypes

,Sales Rep,Sales Rep Id,Company Name,Company Id
0,Jessie Mcallister,97UNNAT790E0WM4N,Chimera-Chasing Casbah,LJKS5NK6788CYMUU
1,Jessie Mcallister,97UNNAT790E0WM4N,Tangential Sheds,36MFTZOYMTAJP1RK
2,Jessie Mcallister,97UNNAT790E0WM4N,Two-Mile Grab,H3JRC7XX7WJAD4ZO
3,Jessie Mcallister,97UNNAT790E0WM4N,Three-Men-And-A-Helper Congo'S,HB25MDZR0MGCQUGX
4,Jessie Mcallister,97UNNAT790E0WM4N,Biophysical Battleground,7RVA8TIVBLBXMNO4


In [28]:
invoices.head(2)

,Order Id,Date,Meal Id,Company Id,Date of Meal,Participants,Meal Price,Type of Meal,Heroes Adjustment
0,839FKFW2LLX4LMBB,2016-05-27,INBUX904GIHI8YBD,LJKS5NK6788CYMUU,2016-05-31 05:00:00+00:00,['David Bishop'],469,Breakfast,False
1,97OX39BGVMHODLJM,2018-09-27,J0MMOOPP709DIDIE,LJKS5NK6788CYMUU,2018-10-01 18:00:00+00:00,['David Bishop'],22,Dinner,False


In [36]:
pd.merge(order_data, invoices) # defaults to only rows with data in common columns.
pd.merge(order_data, invoices, how='left') # uses left df rows and attaches data that exists from merged df
pd.merge(order_data, invoices, how='right') # vice versa
pd.merge(order_data, invoices, how='outer') 
pass


In [42]:
# if merging on a explicit column then other common columns will be suffixed. default suffixes are x and y.
pd.merge(order_data, invoices, on=['Order Id'], suffixes=('_base', '_join'))
pass

In [44]:
#You would typically use the left_on and right_on parameters when the columns are named differently in the two DataFrames.
pd.merge(order_data, invoices.rename(columns={'Order Id':'New Id'}), left_on=['Order Id'], right_on=['New Id'])
pass

Note that map() can also me used to merge. Example in link at top of page.

Summary of combining DataFrames:
1. Use pd.concat to “stack” multiple DataFrame on top or next to each other. By default, the stacking is vertical. To overwrite the default use axis=1. By default, stack will carry over all columns/indices. To limit to common columns/indices use join='inner'.
2. Do not use pd.DataFrame.append as it is only a special, limited case of pd.concat and will only make your code less standardized and less coherent. Instead, use pd.concat.
3. Use pd.merge to combine information from two DataFrames. Merge defaults to an inner join and will infer the columns to merge on from the largest subset of common columns across the DataFrames.
4. Do not use pd.DataFrame.join as it is only a special, limited case of pd.merge and will only make your code less standardized and less coherent. Instead, use pd.merge.
5. Use pd.Series.map as a lookup-esque kind of functionality to get the value for a specific index/key from a series/dictionary.

## Reshaping Dataframes: transpose, melt, pivot, stack, unstack, groupby

In [46]:
invoices.head(5).T # transform (rotate around axis, just like excel)
pass

In [48]:
melt_experiment = pd.merge(
    invoices,
    pd.get_dummies(invoices['Type of Meal']).mul(invoices['Meal Price'].values,axis=0),
    left_index=True,
    right_index=True
)
del melt_experiment['Type of Meal']
del melt_experiment['Meal Price']
melt_experiment.head(2)

,Order Id,Date,Meal Id,Company Id,Date of Meal,Participants,Heroes Adjustment,Breakfast,Dinner,Lunch
0,839FKFW2LLX4LMBB,2016-05-27,INBUX904GIHI8YBD,LJKS5NK6788CYMUU,2016-05-31 05:00:00+00:00,['David Bishop'],False,469,0,0
1,97OX39BGVMHODLJM,2018-09-27,J0MMOOPP709DIDIE,LJKS5NK6788CYMUU,2018-10-01 18:00:00+00:00,['David Bishop'],False,0,22,0


In [78]:
# groupby()
invoices.groupby(['Company Id', 'Type of Meal']).agg({'Meal Price':np.mean})
pass

In [80]:
# pivot()
pd.pivot_table(
    invoices,
    index=['Company Id','Type of Meal'],
    values='Meal Price',
    aggfunc=np.mean,
    margins=True
)
pass

In [ ]:
#stack and unstack come in really handy when rearranging your columns and indices. 
foo.stack()
foo.unstack()
# refer to article link at top for examples